In [1]:
# %% Import necessary libraries
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import torch
import torch.optim as optim
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from preprocessing import Preprocessing
from cnn_model import SpectrogramArrivalCNN
from training import ModelTrainer
from dataloader import DataLoaderHandler

# %% Paths to your data
lunar_catalog_path = 'data/lunar/training/catalogs/apollo12_catalog_GradeA_final.csv'
lunar_data_directory = 'data/lunar/training/data/S12_GradeA/'
lunar_data_images_dir = 'model/model_output/lunar_preprocessed_images/'
save_dir_lunar = lunar_data_images_dir

martian_data_directory = 'data/mars/training/data/'
martian_data_images_dir = 'model/model_output/martian_preprocessed_images/'
save_dir_mars = martian_data_images_dir

# Load lunar catalog
lunar_catalog = pd.read_csv(lunar_catalog_path)

In [2]:
# %% Preprocess lunar data
preprocessor = Preprocessing(save_dir_lunar)
lunar_data, lunar_labels, lunar_arrival_times = preprocessor.preprocess_lunar_data(
    catalog=lunar_catalog, data_dir=lunar_data_directory, combine_images=True
)

Detected Event Arrival Time (relative): 2764.83 s, (absolute): 1970-01-19T00:46:05.495189
Detected Event Arrival Time (relative): 23902.49 s, (absolute): 1970-03-26T06:38:23.055566
Detected Event Arrival Time (relative): 4808.00 s, (absolute): 1970-04-25T01:20:08.196000
Detected Event Arrival Time (relative): 690.11 s, (absolute): 1970-04-26T00:11:30.773208
Detected Event Arrival Time (relative): 39007.70 s, (absolute): 1970-06-15T10:50:08.208113
Detected Event Arrival Time (relative): 72180.08 s, (absolute): 1970-06-26T20:03:00.191472
Detected Event Arrival Time (relative): 18486.19 s, (absolute): 1970-07-20T05:08:06.675679
Detected Event Arrival Time (relative): 18486.19 s, (absolute): 1970-07-20T05:08:06.675679
Detected Event Arrival Time (relative): 665.96 s, (absolute): 1970-09-26T00:11:06.111264
Detected Event Arrival Time (relative): 16960.91 s, (absolute): 1970-10-24T04:42:41.409660
Detected Event Arrival Time (relative): 10203.47 s, (absolute): 1970-11-12T02:50:04.171698
Detec

In [3]:
# %% Convert arrival times to relative time in seconds
reference_time = pd.Timestamp('1970-01-01')
lunar_arrival_times_in_seconds = preprocessor.convert_abs_to_rel_time(lunar_arrival_times, reference_time)

In [4]:
# %% Train-test split on lunar data
lunar_data_train, lunar_data_test, lunar_times_train, lunar_times_test = train_test_split(
    lunar_data, lunar_arrival_times_in_seconds, test_size=0.2, random_state=42
)

In [5]:
# %% Normalize the arrival times
scaler = MinMaxScaler()
lunar_times_train_normalized = scaler.fit_transform(np.array(lunar_times_train).reshape(-1, 1)).flatten()
lunar_times_test_normalized = scaler.transform(np.array(lunar_times_test).reshape(-1, 1)).flatten()

In [6]:
# %% Prepare DataLoader for training and testing
dataloader_handler = DataLoaderHandler(batch_size=32)
lunar_train_loader = dataloader_handler.prepare_data_for_training(lunar_data_train, lunar_times_train_normalized)
lunar_test_loader = dataloader_handler.prepare_data_for_training(lunar_data_test, lunar_times_test_normalized)

In [7]:
# %% Initialize the model and move it to the appropriate device
cnn_model = SpectrogramArrivalCNN()

In [8]:
# %% Set up loss function and optimizer
criterion_time = torch.nn.MSELoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=0.00001)

In [9]:
# %% Train the model on lunar data
trainer = ModelTrainer(cnn_model, criterion_time, optimizer)
trainer.train(lunar_train_loader, num_epochs=20)

/home/aaron/GitHub/bespoke/SIFTER/.venv/lib/python3.11/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/aaron/GitHub/bespoke/SIFTER/.venv/lib/python3.11/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([28])) that is different to the input size (torch.Size([28, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/20], Loss: 0.1975717470049858
Epoch [2/20], Loss: 0.17310839891433716
Epoch [3/20], Loss: 0.16392169892787933
Epoch [4/20], Loss: 0.1622145026922226
Epoch [5/20], Loss: 0.18764102458953857
Epoch [6/20], Loss: 0.19298135489225388
Epoch [7/20], Loss: 0.17319434881210327
Epoch [8/20], Loss: 0.15053576231002808
Epoch [9/20], Loss: 0.15659362077713013
Epoch [10/20], Loss: 0.14627627655863762
Epoch [11/20], Loss: 0.16067910194396973
Epoch [12/20], Loss: 0.1615368202328682
Epoch [13/20], Loss: 0.1408526673913002
Epoch [14/20], Loss: 0.16656813025474548
Epoch [15/20], Loss: 0.12675678730010986
Epoch [16/20], Loss: 0.14464038610458374
Epoch [17/20], Loss: 0.12265098839998245
Epoch [18/20], Loss: 0.147186778485775
Epoch [19/20], Loss: 0.13060923293232918
Epoch [20/20], Loss: 0.13358046114444733


In [10]:
# %% Evaluate the model on the test set
trainer.evaluate(lunar_test_loader, scaler=scaler, reference_time=reference_time)

Saved predicted absolute arrival times to CSV.
Test Loss: 0.12299759685993195


/home/aaron/GitHub/bespoke/SIFTER/.venv/lib/python3.11/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [11]:
# %% Save the trained model
trainer.save_cnn_model('lunar_seismic_cnn_model_full.pth')
trainer.save_cnn_model_state_dict('lunar_seismic_cnn_model_state_dict.pth')

Saved the full model to lunar_seismic_cnn_model_full.pth.
Saved the model's state_dict to lunar_seismic_cnn_model_state_dict.pth.


In [12]:
# %% Load the full pretrained lunar model
cnn_model = torch.load('lunar_seismic_cnn_model_full.pth')
cnn_model.train()  # Set to training mode

/tmp/ipykernel_549311/2437316367.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  cnn_model = torch.load('lunar_seismic_cnn_model_full.pth')


SpectrogramArrivalCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc_time): Linear(in_features=128, out_features=1, bias=True)
  (fc1): Linear(in_features=200704, out_features=128, bias=True)
)

In [13]:
import importlib
import dataloader
from dataloader import DataLoaderHandler

importlib.reload(dataloader)

<module 'dataloader' from '/home/aaron/GitHub/bespoke/SIFTER/dataloader.py'>

In [14]:
# %% Preprocess and self-train on Martian data
preprocessor = Preprocessing(save_dir_mars)
martian_images, _ = preprocessor.preprocess_martian_data(data_dir=martian_data_directory)
martian_data_loader = dataloader_handler.prepare_unlabeled_data_loader(martian_images)

Detected Event Arrival Time (relative): 2143.75 s, (absolute): 2022-01-02T04:35:43.775000
Detected Event Arrival Time (relative): 535.80 s, (absolute): 2022-02-03T08:08:55.809000


In [15]:
# %% Self-training on Martian data
trainer.self_train_on_martian_data(martian_data_loader, criterion_time=criterion_time, num_epochs=10)

Epoch [1/10], Loss Time: 0.0
Epoch [2/10], Loss Time: 0.0
Epoch [3/10], Loss Time: 0.0
Epoch [4/10], Loss Time: 0.0
Epoch [5/10], Loss Time: 0.0
Epoch [6/10], Loss Time: 0.0
Epoch [7/10], Loss Time: 0.0
Epoch [8/10], Loss Time: 0.0
Epoch [9/10], Loss Time: 0.0
Epoch [10/10], Loss Time: 0.0


In [16]:
# %% Save the fine-tuned model after self-training on Martian data
trainer.save_cnn_model('martian_seismic_cnn_model_full.pth')
trainer.save_cnn_model_state_dict('martian_seismic_cnn_model_state_dict.pth')

Saved the full model to martian_seismic_cnn_model_full.pth.
Saved the model's state_dict to martian_seismic_cnn_model_state_dict.pth.


In [18]:
# %% Define the function to evaluate and compute metrics
def evaluate_and_get_metrics(trainer, test_loader):
    """
    Evaluate the model and compute metrics like MSE, MAE, and R-squared on the test set.
    """
    trainer.cnn_model.eval()  # Set to evaluation mode
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in test_loader:
            # Unpack batch depending on the structure returned by the dataloader
            if isinstance(batch, (list, tuple)):
                inputs, time_labels = batch[0], batch[1]
            else:
                inputs = batch
                time_labels = None  # Adjust as per the actual data structure

            # Forward pass
            time_output = trainer.cnn_model(inputs)

            # Collect predictions and actual labels
            all_preds.extend(time_output.cpu().numpy())
            all_labels.extend(time_labels.cpu().numpy())

    # Convert to numpy arrays
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    # Compute regression metrics
    mse = mean_squared_error(all_labels, all_preds)
    mae = mean_absolute_error(all_labels, all_preds)
    r2 = r2_score(all_labels, all_preds)

    # Print metrics
    print(f"Mean Squared Error (MSE): {mse}")
    print(f"Mean Absolute Error (MAE): {mae}")
    print(f"R-squared (R²): {r2}")

    return mse, mae, r2

# %% Evaluate the model and compute metrics on the test set
mse, mae, r2 = evaluate_and_get_metrics(trainer, lunar_test_loader)


Mean Squared Error (MSE): 0.11017212271690369
Mean Absolute Error (MAE): 0.28977087140083313
R-squared (R²): -0.3682965040206909
